In [10]:
import sys

In [11]:
sys.executable

'/n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python3'

In [4]:
from pathlib import Path

In [69]:
python_script_loc = Path('/n/groups/datta/tim_sainburg/datasets/scratch/test_python.py')
bash_script_loc = Path('/n/groups/datta/tim_sainburg/datasets/scratch/test_bash.sh')

In [70]:
slurm_script = f"""#!/usr/bin/env bash
#SBATCH --partition=short
#SBATCH --job-name=test_job
#SBATCH --cpus-per-task=1
#SBATCH --mem=1G
#SBATCH --time=0:01:00

module load gcc/9.2.0

python {python_script_loc.as_posix()}
"""
print(slurm_script)

#!/usr/bin/env bash
#SBATCH --partition=short
#SBATCH --job-name=test_job
#SBATCH --cpus-per-task=1
#SBATCH --mem=1G
#SBATCH --time=0:01:00

module load gcc/9.2.0

python /n/groups/datta/tim_sainburg/datasets/scratch/test_python.py



In [71]:
python_script = "print('Hello World'); import time; time.sleep(60*10)"

In [72]:
# Create directories if they do not exist
python_script_loc.parent.mkdir(parents=True, exist_ok=True)
bash_script_loc.parent.mkdir(parents=True, exist_ok=True)

# Write the Slurm script to file
bash_script_loc.write_text(slurm_script)

# Write the Python script to file
python_script_loc.write_text(python_script)

52

In [73]:
import paramiko

In [74]:
submit_command = f"sbatch {bash_script_loc}"
submit_command

'sbatch /n/groups/datta/tim_sainburg/datasets/scratch/test_bash.sh'

In [75]:
!tail /n/groups/datta/tim_sainburg/datasets/scratch/test_bash.sh

#!/usr/bin/env bash
#SBATCH --partition=short
#SBATCH --job-name=test_job
#SBATCH --cpus-per-task=1
#SBATCH --mem=1G
#SBATCH --time=0:01:00

module load gcc/9.2.0

python /n/groups/datta/tim_sainburg/datasets/scratch/test_python.py


In [76]:
username = 'tis697'
o2_server = 'login.o2.rc.hms.harvard.edu'

In [77]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(o2_server, username=username)
stdin, stdout, stderr = ssh.exec_command(submit_command)
output = stdout.read().decode()
print(output)
ssh.close()

Submitted batch job 43523952



In [78]:
output

'Submitted batch job 43523952\n'

In [79]:
slurm_output = output

In [80]:
slurm_output

'Submitted batch job 43523952\n'

In [81]:
if slurm_output[: len(expected_slurm_output)] != expected_slurm_output:
    raise Exception(f"Error submitting job: {slurm_output}")
job_id = slurm_output[len(expected_slurm_output) :-1]

In [84]:
!scontrol show job {job_id}

JobId=43523952 JobName=test_job
   UserId=tis697(190153) GroupId=tis697(2190153) MCS_label=N/A
   Priority=628392 Nice=0 Account=datta QOS=normal
   JobState=PENDING Reason=None Dependency=(null)
   Requeue=0 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=0:0
   RunTime=00:00:00 TimeLimit=00:01:00 TimeMin=N/A
   SubmitTime=2024-08-07T11:36:12 EligibleTime=2024-08-07T11:36:12
   AccrueTime=2024-08-07T11:36:12
   StartTime=Unknown EndTime=Unknown Deadline=N/A
   SuspendTime=None SecsPreSuspend=0 LastSchedEval=2024-08-07T11:36:12 Scheduler=Main
   Partition=short AllocNode:Sid=login05:20031
   ReqNodeList=(null) ExcNodeList=compute-h-17-[50-55]
   NodeList=
   NumNodes=1 NumCPUs=1 NumTasks=1 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   ReqTRES=cpu=1,mem=1G,node=1,billing=1
   AllocTRES=(null)
   Socks/Node=* NtasksPerN:B:S:C=0:0:*:* CoreSpec=*
   MinCPUsNode=1 MinMemoryNode=1G MinTmpDiskNode=0
   Features=(null) DelayBoot=00:00:00
   OverSubscribe=OK Contiguous=0 Licenses=(null) Network=(null)
   Command=

In [88]:
check_command = f"sacct -j {job_id} --format=JobID,State | grep -E '^[0-9]+ ' | awk '{{print $2}}'"

In [89]:
!{check_command}

RUNNING


In [90]:
!sacct -j {job_id} --format=State --noheader

   RUNNING 
   RUNNING 
   RUNNING 


In [91]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(o2_server, username=username)
stdin, stdout, stderr = ssh.exec_command(check_command)
output = stdout.read().decode()
print(output)
ssh.close()

RUNNING



In [92]:
output[:-1]

'RUNNING'